Basic Trading Operation
=============

This notebook implements trading operations, like authentication, visualization of series and results and the actual trading. Implementation of algorithms should be done in separate modules which can then be imported here.

In [1]:
import base64, hashlib, hmac, time, urllib, json
from urllib.request import urlopen
from getpass import getpass, getuser
import requests
base = 'https://anxpro.com'

In [3]:
def post_request(key, secret, path, data):
    hmac_obj = hmac.new(secret, path + chr(0) + data, hashlib.sha512)
    hmac_sign = base64.b64encode(hmac_obj.digest())

    header = {
        'Content-Type': 'application/json',
        'User-Agent': 'anxv2 based client',
        'Rest-Key': key,
        'Rest-Sign': hmac_sign,
    }

    proxy = urllib2.ProxyHandler({'http': '127.0.0.1:8888'})
    opener = urllib2.build_opener(proxy)
    urllib2.install_opener(opener)

    request = urllib2.Request(base + path, data, header)
    response = urllib2.urlopen(request, data)
    return json.load(response)


def gen_tonce():
    return str(int(time.time() * 1e6))


class ANX:

    def __init__(self, key, secret):
        self.key = key
        self.secret = base64.b64decode(secret)

    def request(self, path, params={}):
        params = dict(params)
        params['tonce'] = gen_tonce()
        # data = urllib.urlencode(params)
        data = json.dumps(params)

        result = post_request(self.key, self.secret, path, data)
        if result['result'] == 'success':
            return result['data']
        else:
            raise Exception(result['result'])

x = ANX('<key>', '<secret>')
result = x.request('api/3/order/list')

Error: Incorrect padding

Getting an API Key
---------------------
Here we will try to generate and get an API key directly through the api. Alternatively, You can generate it directly through the web after logging in to your account at www.anxpro.com.

In [10]:
user = input("enter your email:")
password = getpass("Enter your password:")

enter your email:fccoelho@gmail.com
Enter your password:········


In [30]:

params = {
    'nonce': str(int(time.time() * 1e6)),
    'username': user,
    'password': password,
    'otp': "400345", #fill in from your cell phone,
    'deviceId': 'vaio'
} 
def get_key(params):
    resp = requests.post(base+'/api/3/apiKey', headers=header, data=params)
    return resp

resp = get_key(params)
print(resp.status_code)
print(resp.content)

403
b'{"resultCode":"INVALID_PARAMETERS","timestamp":1451867718729}'
